In [4]:
import pandas as pd
import re
from difflib import SequenceMatcher
df = pd.read_csv("../wikipedia_kyiv_revisions.csv")

toponym_shifts = {
    "Kiev": "Kyiv",
    "Crimea, Ukraine": "Crimea, Russia",
    "Donetsk People's Republic": "Occupied Donetsk",
    "Luhansk People's Republic": "Luhansk, Ukraine",
    "Russian Crimea": "Ukrainian Crimea"
}

# def detect_toponym_changes(before_text, after_text):
#     changes = []
#     for old_name, new_name in toponym_shifts.items():
#         before_text = str(before_text)
#         after_text = str(after_text)
#         if old_name in before_text and new_name in after_text:
#             changes.append(f"'{old_name}' → '{new_name}'")
#         elif new_name in before_text and old_name in after_text:
#             changes.append(f"'{new_name}' → '{old_name}'")  # If reversed
#     return ", ".join(changes) if changes else ""

# df["toponym_changes"] = df.apply(lambda row: detect_toponym_changes(row["before_content"], row["after_content"]), axis=1)

# df_changes = df[df["toponym_changes"] != ""]

# df_changes.to_csv("wikipedia_toponymy_changes.csv", index=False, encoding="utf-8")

# df_changes

,revision_id,previous_id,timestamp,user,comment,content,before_content,after_content,diff,toponym_changes
6,24507,24434,"time.struct_time(tm_year=2002, tm_mon=3, tm_md...",128.100.141.23,*,"The capital and largest city of [[Ukraine]], <...","The capital and largest city of [[Ukraine]], <...","The capital and largest city of [[Ukraine]], <...","--- \n+++ \n@@ -2,7 +2,7 @@\n \n The city was ...",'Kiev' → 'Kyiv'
7,99199,24507,"time.struct_time(tm_year=2002, tm_mon=3, tm_md...",David Parker,*,"The capital and largest city of [[Ukraine]], <...","The capital and largest city of [[Ukraine]], <...","The capital and largest city of [[Ukraine]], <...","--- \n+++ \n@@ -1,9 +1,7 @@\n-The capital and ...",'Kiev' → 'Kyiv'
8,109429,99199,"time.struct_time(tm_year=2002, tm_mon=6, tm_md...",The Epopt,links,"The capital and largest city of [[Ukraine]], <...","The capital and largest city of [[Ukraine]], <...","The capital and largest city of [[Ukraine]], <...","--- \n+++ \n@@ -1,7 +1,7 @@\n The capital and ...",'Kiev' → 'Kyiv'
9,171959,109429,"time.struct_time(tm_year=2002, tm_mon=7, tm_md...",Mav,reversing article/redirect order per naming co...,"The capital and largest city of [[Ukraine]], <...","The capital and largest city of [[Ukraine]], <...","The capital and largest city of [[Ukraine]], <...",NaN,'Kiev' → 'Kyiv'
10,171958,171959,"time.struct_time(tm_year=2002, tm_mon=7, tm_md...",Mav,"#REDIRECT [[Kiev, Ukraine]]","#REDIRECT [[Kiev, Ukraine]]","The capital and largest city of [[Ukraine]], <...","#REDIRECT [[Kiev, Ukraine]]","--- \n+++ \n@@ -1,7 +1 @@\n-The capital and la...",'Kyiv' → 'Kiev'
...,...,...,...,...,...,...,...,...,...,...
6016,1271551319,1270738625,"time.struct_time(tm_year=2025, tm_mon=1, tm_md...",Aerrapc,"""previously spelled"", instead of ""also""",{{Short description|Capital of Ukraine}}\n{{Ab...,{{Short description|Capital of Ukraine}}\n{{Ab...,{{Short description|Capital of Ukraine}}\n{{Ab...,"--- \n+++ \n@@ -132,7 +132,7 @@\n <!-- website...",'Kiev' → 'Kyiv'
6017,1271595101,1271551319,"time.struct_time(tm_year=2025, tm_mon=1, tm_md...",Mellk,Undid revision [[Special:Diff/1271551319|12715...,{{Short description|Capital of Ukraine}}\n{{Ab...,{{Short description|Capital of Ukraine}}\n{{Ab...,{{Short description|Capital of Ukraine}}\n{{Ab...,"--- \n+++ \n@@ -132,7 +132,7 @@\n <!-- website...",'Kiev' → 'Kyiv'
6018,1271666506,1271595101,"time.struct_time(tm_year=2025, tm_mon=1, tm_md...",Tigerdude9,/* Independence */,{{Short description|Capital of Ukraine}}\n{{Ab...,{{Short description|Capital of Ukraine}}\n{{Ab...,{{Short description|Capital of Ukraine}}\n{{Ab...,"--- \n+++ \n@@ -223,7 +223,7 @@\n Kyiv recover...",'Kiev' → 'Kyiv'
6019,1277706684,1271666506,"time.struct_time(tm_year=2025, tm_mon=2, tm_md...",CapLiber,NaN,{{Short description|Capital of Ukraine}}\n{{Ab...,{{Short description|Capital of Ukraine}}\n{{Ab...,{{Short description|Capital of Ukraine}}\n{{Ab...,"--- \n+++ \n@@ -7,7 +7,7 @@\n | name ...",'Kiev' → 'Kyiv'


In [8]:
df_changes.iloc[6]

revision_id                                                   485133
previous_id                                                   456577
timestamp          time.struct_time(tm_year=2002, tm_mon=12, tm_m...
user                                                  203.109.254.58
comment                                                          NaN
content            The capital and largest city of [[Ukraine]], <...
before_content     The capital and largest city of [[Ukraine]], <...
after_content      The capital and largest city of [[Ukraine]], <...
diff               --- \n+++ \n@@ -1,7 +1,7 @@\n-The capital and ...
toponym_changes                                      'Kiev' → 'Kyiv'
Name: 12, dtype: object

In [6]:
df_changes.iloc[6].before_content

"The capital and largest city of [[Ukraine]], <b>Kiev</b> (<i>Kyiv</i> in Ukrainian and <i>Kiyev</i> in Russian) contains around 2.6 million inhabitants. \r\n\r\nThe city was probably founded in the [[5th century]] CE and functioned as a trade post between [[Constantinople]] and [[Scandinavia]].  The Gothic historian [[Jordanes]] recorded the trading town of <i>Danapirstadir</i>.  As the region came under Slav rule the town became known as Kiev. \r\n\r\nFrom [[882]] until [[1169]] Kiev was the capital of the principal East Slavic state, known as [[Kievan Rus|Kievan Rus']] (or Kyivan Rus'). Devastated by the invading [[Mongols]] in [[1240]], it subsequently passed under the rule of [[Lithuania]] ([[1362]]), [[Poland]] ([[1569]]), a short-lived Ukrainian [[Cossack]] state ([[1648]]), and [[Russia]] ([[1654]]/67). After 57 years as the capital of the Ukrainian Soviet Socialist Republic of the [[Soviet Union]], Kiev in [[1991]] became the capital of independent [[Ukraine]]. \r\n\r\nThe chu

In [7]:
df_changes.iloc[6].after_content

'The capital and largest city of [[Ukraine]], <b>Kiev</b> (<i>Kyiv</i> in Ukrainian and <i>Kiyev</i> in Russian) has around 2.6 million inhabitants. \r\n\r\nThe city was probably founded in the [[5th century]] CE and functioned as a trade post between [[Constantinople]] and [[Scandinavia]].  The Gothic historian [[Jordanes]] recorded the trading town of <i>Danapirstadir</i>.  As the region came under Slav rule the town became known as Kiev. (Legend speaks of a founder-figure named \'\'Kiy\'\'.)\r\n\r\nFrom [[882]] until [[1169]] Kiev was the capital of the principal [[Varangian]]/East-Slavic state, known as [[Kievan Rus|Kievan Rus\']] (or Kyivan Rus\'). Devastated by the invading [[Mongols]] in [[1240]], it subsequently passed under the rule of [[Lithuania]] ([[1362]]), [[Poland]] ([[1569]]), a short-lived Ukrainian [[Cossack]] state ([[1648]]), and [[Russia]] ([[1654]]/67). After 57 years as the capital of the Ukrainian Soviet Socialist Republic of the [[Soviet Union]], Kiev in [[1991

In [17]:
import getpass
if "OPENAI_API_KEY_PROJ" in os.environ:
    OPENAI_API_KEY = os.environ["OPENAI_API_KEY_PROJ"]
else:
    OPENAI_API_KEY = getpass.getpass("Please enter your OpenAI API key: ")

Please enter your OpenAI API key:  ········


In [18]:
df = pd.read_csv("../wikipedia_kyiv_revisions.csv")
df.head()

,revision_id,previous_id,timestamp,user,comment,content,before_content,after_content,diff
0,17179,0,"time.struct_time(tm_year=2002, tm_mon=2, tm_md...",Jeronimo,NaN,Capital and largest city of the [[Ukraine]] (2...,NaN,Capital and largest city of the [[Ukraine]] (2...,"--- \n+++ \n@@ -0,0 +1,3 @@\n+Capital and larg..."
1,17195,17179,"time.struct_time(tm_year=2002, tm_mon=2, tm_md...",66.47.62.78,*conversions,Capital and largest city of the [[Ukraine]] (2...,Capital and largest city of the [[Ukraine]] (2...,Capital and largest city of the [[Ukraine]] (2...,"--- \n+++ \n@@ -1,3 +1,5 @@\n Capital and larg..."
2,17221,17195,"time.struct_time(tm_year=2002, tm_mon=2, tm_md...",MichaelTinkler,removing Ansgar. Ansgar was c. 800 and in Den...,Capital and largest city of the [[Ukraine]] (2...,Capital and largest city of the [[Ukraine]] (2...,Capital and largest city of the [[Ukraine]] (2...,"--- \n+++ \n@@ -1,5 +1,5 @@\n-Capital and larg..."
3,17232,17221,"time.struct_time(tm_year=2002, tm_mon=2, tm_md...",David Parker,"City, not state. Remove peripheral nonsense.","The capital and largest city of [[Ukraine]], <...",Capital and largest city of the [[Ukraine]] (2...,"The capital and largest city of [[Ukraine]], <...","--- \n+++ \n@@ -1,5 +1,7 @@\n-Capital and larg..."
4,17238,17232,"time.struct_time(tm_year=2002, tm_mon=2, tm_md...",David Parker,Capital 1934,"The capital and largest city of [[Ukraine]], <...","The capital and largest city of [[Ukraine]], <...","The capital and largest city of [[Ukraine]], <...","--- \n+++ \n@@ -1,7 +1,7 @@\n-The capital and ..."


In [ ]:
import openai
import pandas as pd
import os
from openai import OpenAI
import json

client = OpenAI(api_key=OPENAI_API_KEY)

# Function to call OpenAI API with structured prompting
def detect_toponymy_shift(before_text, after_text):
    prompt = f"""
You are an expert in geopolitics and historical linguistics. Your task is to detect changes in place names in Wikipedia edits and analyze their significance. 

### Examples:
1. 'Kiev' → 'Kyiv' (Modern Ukrainian transliteration)
2. 'Crimea, Ukraine' → 'Crimea, Russia' (Reflects Russian annexation)
3. 'Donetsk People's Republic' → 'Occupied Donetsk' (Political framing)

Now analyze the following texts:

#### Before Edit:
"{before_text}"

#### After Edit:
"{after_text}"

### Task:
- Identify any place name changes.
- Determine if the change reflects political, historical, or linguistic shifts.
- Explain the significance of the change in the context of Russia-Ukraine conflicts.

Provide a structured JSON response:

```json
{{
  "detected_changes": [
    {{"before": "PLACE_NAME_BEFORE", "after": "PLACE_NAME_AFTER"}}
  ],
  "type_of_change": "Political/Historical/Linguistic",
  "significance": "EXPLANATION"
}}
    """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
    #     response_format={
    #         "type": "json_schema",
    #         "json_schema": {
    #             "name": "toponymy_shifts",
    #             "schema": {
    #             "type": "object",
    #             "properties": {
    #                 "detected_changes": {
    #                     "type": "array",
    #                     "items": {
    #                         "type": "object",
    #                         "properties": {
    #                             "before": {"type": "string"},
    #                             "after": {"type": "string"}
    #                         }
    #                     }
    #                 },
    #                 "type_of_change": {"type": "string"},
    #                 "significance": {"type": "string"}
    #             },
    #             "required": ["detected_changes", "type_of_change", "significance"]
    #         }
    #     }
    # }
    )

    x = response.choices[0].message.content
    # json_output = json.loads(x)
    print(x)
    print('-'*30)
    return x
    
df["toponymy_analysis"] = df.apply(lambda row: detect_toponymy_shift(row["before_content"], row["after_content"]), axis=1)

df.to_csv("wikipedia_toponymy_analysis.csv", index=False, encoding="utf-8")
df

```json
{
  "detected_changes": [],
  "type_of_change": "N/A",
  "significance": "No place name changes were detected from the provided texts. However, the description (after edit) pertains to the city of Kyiv in Ukraine but the specific name 'Kyiv' is not mentioned. The description emphasizes the Ukrainian transliteration instead of the Russian one which may signify recognition or endorsement of Ukraine's sovereignty and independence in the context of Russia-Ukraine conflicts."
}
```
------------------------------
The provided texts are largely the same, and there seems to be no changes in place names. The changes that appear in the text are added details related to the history, religion, and notable figures of the considered location. The information does not appear to be directly related to Russia-Ukraine conflicts. 

As a result, the JSON response would be 

```json
{
  "detected_changes": [],
  "type_of_change": "None",
  "significance": "No significant changes in place names dete